<a href="https://colab.research.google.com/github/LammyG/HDSC-Machine-Learning-Regression-/blob/main/ML_Classification_Codes_Stage_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
!pip install -U openpyxl
!pip install catboost
!pip install fitter
from fitter import Fitter, get_common_distributions, get_distributions

     |████████████████████████████████| 76.1 MB 1.2 MB/s 
     |████████████████████████████████| 47 kB 3.8 MB/s 
  Created wheel for fitter: filename=fitter-1.4.0-py3-none-any.whl size=25026 sha256=b2b65c3976a457b6bf2e9fe5957cc8d7d5a89a598d943a4d41336c7df1981f57
  Stored in directory: /root/.cache/pip/wheels/e1/98/16/e5263962f94fbfaad79902aa94652516caccc1f1d51509e853
  Created wheel for easydev: filename=easydev-0.12.0-py3-none-any.whl size=64232 sha256=38c313a8e549dad77fc5ecb9026600083c108f9e702834824da7d1c351f3a64e
  Stored in directory: /root/.cache/pip/wheels/82/ab/83/fdfc4017ea44a585b6754752cc5f63f2d0d63fcc1317e7174b
Successfully built fitter easydev


In [2]:
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#data url
url = ('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')

In [4]:
df = pd.read_csv(url, low_memory=False)
df.head()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [5]:
import sklearn as sn
import sklearn.preprocessing as pp
import sklearn.linear_model as Lm
import sklearn.model_selection as Ms
import sklearn.metrics as snms


In [6]:
#check distribution of target variable
#prints
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [7]:
#check for missing values in the dataset
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [9]:
import sklearn.utils
df['QScore'].value_counts()
df.isna().sum()
#for simplicity, we will drop the rows with missing values.
df = df.dropna()
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [10]:
#lets do value count again
df.QScore.value_counts()

3A    51473
2A      224
1A       16
Name: QScore, dtype: int64

In [11]:

#reset the dataframe index
df = df.reset_index(drop=True)

In [12]:
#then convert to binary classification by combining 2A and 1A
df['QScore'] = df['QScore'].replace(['1A'], '2A')
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [13]:
#seperating the target variable and
#selecting some samples
df_2A = df[df.QScore=='2A']
df_3A = df[df.QScore=='3A'].sample(350)
data_df = df_2A.append(df_3A)
data_df

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
1096,Algeria,2016,4,AreaPerCap,2.072989e-01,8.112722e-01,0.048357265,0.022585,2.998367e-02,0.000000,1.119497e+00,2A
1097,Algeria,2016,4,AreaTotHA,8.417600e+06,3.294260e+07,1963600,917100.000000,1.217520e+06,0.000000,4.545842e+07,2A
1098,Algeria,2016,4,BiocapPerCap,2.021916e-01,2.636077e-01,0.027166736,0.007948,2.924496e-02,0.000000,5.301590e-01,2A
1099,Algeria,2016,4,BiocapTotGHA,8.210214e+06,1.070408e+07,1103135.245,322736.916200,1.187524e+06,0.000000,2.152769e+07,2A
1100,Algeria,2016,4,EFConsPerCap,6.280528e-01,1.810332e-01,0.162800822,0.014729,2.924496e-02,1.391455,2.407316e+00,2A
...,...,...,...,...,...,...,...,...,...,...,...,...
11414,Cuba,1984,49,EFProdPerCap,4.150308e-01,1.000905e-01,0.130761699,0.172790,3.659940e-02,1.233352,2.088625e+00,3A
42819,"Tanzania, United Republic of",2008,215,BiocapPerCap,3.430778e-01,3.740505e-01,0.193148826,0.067748,4.556231e-02,0.000000,1.023587e+00,3A
6847,Burundi,1973,29,EFProdTotGHA,1.790073e+06,6.725663e+05,1559485.15,35817.608240,1.277430e+05,26565.285310,4.212251e+06,3A
521,Afghanistan,2001,2,AreaTotHA,7.753000e+06,3.000000e+07,1350000,0.000000,9.422190e+05,0.000000,4.004522e+07,3A


In [14]:
#to reshuffle the dataset for randomness
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df.QScore.value_counts()
#prints

3A    350
2A    240
Name: QScore, dtype: int64

In [15]:
#check the datatype of the dataset
data_df.dtypes

# from the above, we will be dropping country code, country and year
#because they are ambiguous to what we want to predict
#and also we will encode the record feature

country            object
year                int64
country_code        int64
record             object
crop_land         float64
grazing_land      float64
forest_land        object
fishing_ground    float64
built_up_land     float64
carbon            float64
total             float64
QScore             object
dtype: object

In [16]:
#on of the features is categorical, so we need to encode it ahead
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data_df.record = encoder.fit_transform(data_df.record)

In [17]:
#preview the encoded features
data_df.record

0      5
1      6
2      5
3      0
4      5
      ..
585    2
586    0
587    5
588    6
589    6
Name: record, Length: 590, dtype: int64

In [18]:
#more preprocessing
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
X = data_df.drop(columns='QScore')
y = data_df['QScore']

In [19]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y_train.value_counts()

#installing imblearn module (uncomment the pip command to install imblearn)
#!pip install imblearn

#encode categorical variable
#from sklearn.preprocessing import LabelEncoder
#encoder = LabelEncoder()
#x_train.record = encoder.fit_transform(x_train.record)
#x_test.record = encoder.transform(x_test.record)

#the encoded feature
x_train.record

285    7
113    3
18     7
76     7
206    4
      ..
277    4
9      3
359    7
192    1
559    6
Name: record, Length: 413, dtype: int64

In [20]:
#for the imbalance in the class distribution, we use SMOTE on the training data to handle this
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train)
y_train.value_counts()

3A    240
2A    173
Name: QScore, dtype: int64

In [21]:
#min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']

In [22]:
x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.fit_transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [23]:
#logistic regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression()

In [24]:
#measuring classification performance


In [25]:
#cross validation score
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv=5, scoring='f1_macro')
scores

array([0.46730497, 0.53170732, 0.48952794, 0.55299335, 0.49975186])

In [26]:
#confusion matrix
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=['2A', '3A'])
cnf_mat

array([[33, 34],
       [53, 57]])

In [27]:
#METRICS

#Accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print('Accuracy:{}'.format(round(accuracy*100), 2))
#prints

Accuracy:51


In [28]:
#Precision
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Precision: {}'.format(round(precision*100), 2))
#prints



Precision: 38


In [29]:
#Recall
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('Recall: {}'.format(round(recall*100), 2))
#prints

Recall: 49


In [34]:
#F1-score
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label='2A')
print('F1: {}'.format(round(f1*100), 2))
#prints

F1: 43


In [58]:

#K-Fold cross validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.split(normalised_train_df)
f1_scores = []

#run for every split
for train_index, text_index in kf.split(normalised_train_df):
  x_train, x_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
  y_train, y_test = y_balanced[train_index], y_balanced[test_index]
  model = LogisticRegression().fit(x_train, y_train)
  #save result to list
  f1_scores.append(f1_score(y_true = y_test2, y_pred = model.predict(x_test), pos_label='2A')*100)

  f1_scores


In [59]:
#Stratified KFold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
sk_f1_scores = []
#run for every split
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
  x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
  y_train, y_test3 = y_balanced[train_index], y_balanced[test_index]
  model = LogisticRegression().fit(x_train, y_train)
  #save result to list
  sk_f1_scores.append(f1_score(y_true = y_test, y_pred = model.predict(x_test), pos_label = '2A')*100)
  sk_f1_scores

In [60]:
from numpy.core.numeric import cross
#LeaveOneOut
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo, scoring='f1_macro')
average_score = scores.mean() * 100
average_score

51.87500000000001

In [61]:
#Tree-based method and the support vector machine
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)

DecisionTreeClassifier()

In [62]:
#make precision with decision tree
dec_tree_pred = dec_tree.predict(normalised_test_df)

In [91]:
#preview result
dec_tree_pred

array(['3A', '3A', '2A', '3A', '3A', '3A', '3A', '2A', '3A', '3A', '2A',
       '3A', '3A', '2A', '3A', '3A', '3A', '3A', '3A', '3A', '3A', '2A',
       '2A', '3A', '2A', '3A', '3A', '3A', '3A', '3A', '2A', '2A', '3A',
       '3A', '3A', '2A', '3A', '3A', '3A', '3A', '3A', '2A', '3A', '3A',
       '3A', '3A', '3A', '2A', '2A', '2A', '2A', '2A', '3A', '2A', '2A',
       '3A', '3A', '3A', '2A', '3A', '2A', '3A', '3A', '3A', '3A', '2A',
       '3A', '2A', '2A', '3A', '3A', '2A', '3A', '2A', '3A', '3A', '3A',
       '2A', '3A', '2A', '3A', '3A', '2A', '3A', '3A', '2A', '2A', '3A',
       '2A', '2A', '2A', '2A', '3A', '2A', '2A', '3A', '2A', '3A', '2A',
       '2A', '2A', '3A', '3A', '2A', '2A', '2A', '2A', '2A', '2A', '2A',
       '3A', '3A', '2A', '3A', '3A', '3A', '2A', '2A', '2A', '3A', '3A',
       '2A', '2A', '2A', '3A', '3A', '3A', '3A', '3A', '2A', '3A', '3A',
       '2A', '3A', '3A', '2A', '3A', '2A', '3A', '3A', '2A', '3A', '3A',
       '3A', '3A', '3A', '2A', '3A', '2A', '2A', '2

In [102]:
dec_tree_pred.shape

(177,)

In [69]:
y_test.shape

(96,)

In [70]:
new_predictions.shape

(177,)

In [103]:
normalised_test_df.shape

(177, 8)